# Quantum Optimization Run

This notebook executes a single run of the quantum optimization pipeline. The parameters for the run are specified in a YAML file.

In [ ]:
import json
import logging
import os
import sys

import yaml

from pipeline.main import single_run

logger = logging.getLogger("pipeline_logger")


def configure_logger():

    # Prevent duplicate handlers if the cell is run multiple times
    if not logger.handlers:
        logger.setLevel(logging.INFO)
        
        ch = logging.StreamHandler(sys.stdout)
        ch.setLevel(logging.INFO)
        
        formatter = logging.Formatter('[%(asctime)s] %(levelname)s: %(message)s')
        ch.setFormatter(formatter)
        
        logger.addHandler(ch)


def read_parameters(parameter_path: str) -> dict:

    if not os.path.isfile(parameter_path):
        logger.error(f"Error: file '{parameter_path}' does not exist.")
        return None

    try:

        with open(parameter_path, 'r') as file:
            return yaml.safe_load(file)

    except Exception as e:

        logger.error(f"Error reading parameters file: {e}")
        return None

In [ ]:
configure_logger()

# In a notebook, we can't use sys.argv. 
# Instead, specify the path to the parameter file directly.
# You can change this to any valid parameter file.
parameter_path = 'params.yaml'

logger.info(f"reading parameters file: '{parameter_path}'")
parameter_dict = read_parameters(parameter_path)

if parameter_dict:
    output_folder = parameter_dict.get('output_folder', '.')
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    output_dict = single_run(parameter_dict)

    output_file_path = os.path.join(output_folder, "output.json")
    logger.info(f"writing output file: {output_file_path}")
    with open(output_file_path, "w") as f:
        json.dump(output_dict, f, indent=4)
else:
    logger.error("Failed to read parameter file. Halting execution.")